<a href="https://colab.research.google.com/github/lescardone/deep-learning-project/blob/main/CNN_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, shutil
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [ ]:
# Directories for our training,
# validation and test splits
base_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small'

train_dir = os.path.join(base_dir, 'train')

validation_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'test')

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 1000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.applications import MobileNet
conv_mobile = MobileNet(weights='imagenet',
                        include_top=False,
                        input_shape=(224,224,3))

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 1024))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_mobile.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels, generator

In [ ]:
train_features_m, train_labels_m, train_generator_m = extract_features(train_dir, 1000)
validation_features_m, validation_labels_m, validation_generator_m = extract_features(validation_dir, 600)
test_features_m, test_labels_m, test_generator_m = extract_features(test_dir, 652)

Found 1000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 652 images belonging to 2 classes.


In [ ]:
train_features_m = np.reshape(train_features_m, (1000, 7 * 7 * 1024))
validation_features_m = np.reshape(validation_features_m, (600, 7 * 7 * 1024))
test_features_m = np.reshape(test_features_m, (652, 7 * 7 * 1024))

In [ ]:
from keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam


In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=(7 * 7 * 1024)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features_m, train_labels_m,
                    epochs=50,
                    batch_size=20,
                    validation_data=(validation_features_m, validation_labels_m))

In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=(7 * 7 * 1024)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', input_dim=(7 * 7 * 1024)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', input_dim=(7 * 7 * 1024)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features_m, train_labels_m,
                    epochs=20,
                    batch_size=20,
                    validation_data=(validation_features_m, validation_labels_m))

Epoch 1/20
50/50 [==============================] - 5s 80ms/step - loss: 1.4960 - acc: 0.4753 - val_loss: 0.6767 - val_acc: 0.5700
Epoch 2/20
50/50 [==============================] - 4s 73ms/step - loss: 0.9763 - acc: 0.5626 - val_loss: 0.6598 - val_acc: 0.5967
Epoch 3/20
50/50 [==============================] - 4s 73ms/step - loss: 0.8556 - acc: 0.5410 - val_loss: 0.6357 - val_acc: 0.6383
Epoch 4/20
50/50 [==============================] - 4s 72ms/step - loss: 0.8170 - acc: 0.5399 - val_loss: 0.6190 - val_acc: 0.6983
Epoch 5/20
50/50 [==============================] - 4s 73ms/step - loss: 0.7606 - acc: 0.5812 - val_loss: 0.6104 - val_acc: 0.6700
Epoch 6/20
50/50 [==============================] - 4s 73ms/step - loss: 0.6413 - acc: 0.6636 - val_loss: 0.5934 - val_acc: 0.7400
Epoch 7/20
50/50 [==============================] - 4s 72ms/step - loss: 0.6007 - acc: 0.6806 - val_loss: 0.5804 - val_acc: 0.7200
Epoch 8/20
50/50 [==============================] - 4s 73ms/step - loss: 0.5935 - a

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels, generator

train_features, train_labels, train_generator = extract_features(train_dir, 1000)
validation_features, validation_labels, validation_generator = extract_features(validation_dir, 600)
test_features, test_labels, test_generator = extract_features(test_dir, 652)

Found 1000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 652 images belonging to 2 classes.


In [ ]:
train_features = np.reshape(train_features, (1000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (600, 4 * 4 * 512))
test_features = np.reshape(test_features, (652, 4 * 4 * 512))

In [ ]:
from keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_featbures, train_labels,
                    epochs=50,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
model.save('happy_and_sad_small_transfer_mobile_adam_1.h5')